## Install dependencies

In [1]:
!pip install -U sagemaker==2.192.1

## Build Custom Container
First we augment the Dockerfile for SageMaker.

In [23]:
sm_entry_stmt = """
# Text Generation Inference base env
ENV HUGGINGFACE_HUB_CACHE=/tmp \
    HF_HUB_ENABLE_HF_TRANSFER=1 \
    PORT=80
COPY sagemaker-entrypoint.sh entrypoint.sh
RUN chmod +x entrypoint.sh

ENTRYPOINT ["./entrypoint.sh"]
CMD [ "" ]
"""

In [24]:
with open("../tgi-custom/Dockerfile", "r") as fin:
    docker_content = fin.read()

sm_docker_cotent = docker_content + sm_entry_stmt

with open("Dockerfile_sm", "w") as fout:
    fout.write(sm_docker_cotent)

Then we build the image. This could take 10 minutes, so feel free to run it directly in the terminal in case the notebook cell times out.  

**Important Note** - Please ensure the `ROLE` has sufficient permission to push Docker images to Elastic Container Registry.

In [1]:
!cp -r ../tgi-custom/vllm ./vllm

In [26]:
REPO_NAME = "mistrallite-tgi110-ecr"

In [27]:
!bash sm_build.sh {REPO_NAME}

REPO_NAME: mistrallite-tgi110-ecr
REGION: us-west-2
ACCOUNT_ID: 161422014849
1.1.0: Pulling from huggingface/text-generation-inference
Digest: sha256:553a1397345e37bc5c6c9eb2300ee46c7dd5c55937c245871608f903f7f71d7d
Status: Image is up to date for ghcr.io/huggingface/text-generation-inference:1.1.0
ghcr.io/huggingface/text-generation-inference:1.1.0
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-west-2:161422014849:repository/mistrallite-tgi110-ecr",
            "registryId": "161422014849",
            "repositoryName": "mistrallite-tgi110-ecr",
            "repositoryUri": "161422014849.dkr.ecr.us-west-2.amazonaws.com/mistrallite-tgi110-ecr",
            "createdAt": 1697516781.0,
            "imageTagMutability": "

## Deploy SageMaker Endpoint

In [28]:
import boto3
import json
import sagemaker
import time

from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

def get_aws_region():
    # Get the current AWS region from the default session
    session = boto3.session.Session()
    return session.region_name

def get_aws_account_id():
    # Get the current AWS account ID from the default session
    sts_client = boto3.client("sts")
    response = sts_client.get_caller_identity()
    return response["Account"]

REGION = get_aws_region()
ACCOUNT_ID = get_aws_account_id()

role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [29]:
image_uri = f"{ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/{REPO_NAME}"
image_uri

'161422014849.dkr.ecr.us-west-2.amazonaws.com/mistrallite-tgi110-ecr'

In [30]:
model_name = "MistralLite-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

instance_type = "ml.g5.2xlarge"
num_gpu = 1
max_input_length = 24000
max_total_tokens = 24576

hub = {
    'HF_MODEL_ID':'amazon/MistralLite',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS': json.dumps(num_gpu),
    "MAX_INPUT_LENGTH": json.dumps(max_input_length),
    "MAX_TOTAL_TOKENS": json.dumps(max_total_tokens),
    "MAX_BATCH_PREFILL_TOKENS": json.dumps(max_total_tokens),
    "MAX_BATCH_TOTAL_TOKENS":  json.dumps(max_total_tokens),
    "DTYPE": 'bfloat16',
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)
predictor = model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  endpoint_name=model_name,   
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
----------------!

## Perform Inference

### Sagemaker SDK

In [31]:
input_data = {
  "inputs": "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>",
  "parameters": {
    "do_sample": False,
    "max_new_tokens": 400,
    "return_full_text": False,
    #"typical_p": 0.2,
    #"temperature":None,
    #"truncate":None,
    #"seed": 1,
  }
}
result = predictor.predict(input_data)[0]["generated_text"]
print(result)

There are several challenges to supporting a long context for LLMs, including:

1. Memory Limitations: LLMs are limited in the amount of data they can store in memory, which can limit the length of the context they can support.

2. Computational Complexity: Generating a response for a long context can be computationally expensive, especially for larger models.

3. Training Data: Training an LLM to support a long context requires a large amount of training data, which can be difficult and time-consuming to collect.

4. Bias and Unanswerable Questions: As the context gets longer, there is a greater risk of bias and unanswerable questions, which can be difficult for the model to handle.

5. Human Evaluation: Evaluating the quality of responses for a long context can be challenging, as it may require human evaluators to read and understand a large amount of text.

6. Scalability: As the context gets longer, it becomes more difficult to scale the model to handle the increased computational 

### boto3

In [32]:
import boto3
import json
def call_endpoint(client, prompt, endpoint_name, paramters):
    client = boto3.client("sagemaker-runtime")
    payload = {"inputs": prompt,
               "parameters": parameters}
    response = client.invoke_endpoint(EndpointName=endpoint_name,
                                      Body=json.dumps(payload), 
                                      ContentType="application/json")
    output = json.loads(response["Body"].read().decode())
    result = output[0]["generated_text"]
    return result

client = boto3.client("sagemaker-runtime")
parameters = {
    "do_sample": False,
    "max_new_tokens": 400,
    "return_full_text": False,
    #"typical_p": 0.2,
    #"temperature":None,
    #"truncate":None,
    #"seed": 10,
}
endpoint_name = predictor.endpoint_name
prompt = "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>"
result = call_endpoint(client, prompt, endpoint_name, parameters)
print(result)

There are several challenges to supporting a long context for LLMs, including:

1. Memory Limitations: LLMs are limited in the amount of data they can store in memory, which can limit the length of the context they can support.

2. Computational Complexity: Generating a response for a long context can be computationally expensive, especially for larger models.

3. Training Data: Training an LLM to support a long context requires a large amount of training data, which can be difficult and time-consuming to collect.

4. Bias and Unanswerable Questions: As the context gets longer, there is a greater risk of bias and unanswerable questions, which can be difficult for the model to handle.

5. Human Evaluation: Evaluating the quality of responses for a long context can be challenging, as it may require human evaluators to read and understand a large amount of text.

6. Scalability: As the context gets longer, it becomes more difficult to scale the model to handle the increased computational 

Try the long context of over 13,400 tokens, which are copied from [Amazon Aurora FAQs](https://aws.amazon.com/rds/aurora/faqs/)

In [33]:
with open("../example_long_ctx.txt", "r") as fin:
    task_instruction = fin.read()
    task_instruction = task_instruction.format(
        my_question="please tell me how does pgvector help with Generative AI and give me some examples."
    )
prompt = f"<|prompter|>{task_instruction}</s><|assistant|>"
result = call_endpoint(client, prompt, endpoint_name, parameters)
print(result)

pgvector is an open-source extension for PostgreSQL supported by Amazon Aurora PostgreSQL-Compatible Edition.

You can use pgvector to store, search, index, and query billions of embeddings that are generated from machine learning (ML) and artificial intelligence (AI) models in your database, such as those from Amazon Bedrock (limited preview) or Amazon SageMaker. A vector embedding is a numerical representation that represents the semantic meaning of content such as text, images, and video.

With pgvector, you can query embeddings in your Aurora PostgreSQL database to perform efficient semantic similarity searches of these data types, represented as vectors, combined with other tabular data in Aurora. This enables the use of generative AI and other AI/ML systems for new types of applications such as personalized recommendations based on similar text descriptions or images, candidate match based on interview notes, customer service next best action recommendations based on successful t